# Text Classifier

In [11]:
import pandas as pd
import string
import nltk
import numpy as np
import keras
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras import optimizers
from keras.layers import Input
from keras.callbacks import EarlyStopping

In [12]:
data = pd.read_csv(r"E:\EEG DATA\train_ordered\02_tcp_le\Text_data\text_meta.csv")

In [13]:
data.head()

,Unnamed: 0,edf_location,label,report
0,0,E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...,0,CLINICAL HISTORY 55yearold righthanded woman C...
1,1,E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...,1,CLINICAL HISTORY 27yearold woman focal scleros...
2,2,E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...,0,CLINICAL HISTORY 23yearold male history seizur...
3,3,E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...,0,CLINICAL HISTORY 56 year old male small lacuna...
4,4,E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...,0,CLINICAL HISTORY 56 year old male small lacuna...


In [14]:
#All the basic pre-processing: Stopword removal, punctuation removal, stemming, lower casing
data['report'] = data['report'].progress_apply(lambda s: str(s))

from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))
def text_process(text):
    #String punctuation provides all the necessary checks
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
data['report'] = data['report'].progress_apply(lambda txt: str(txt))

exclude = set(string.punctuation)
def punct_remove(text):
    text = ''.join(ch for ch in text if ch not in exclude)
    return text
data['report'] = data['report'].progress_apply(lambda s: punct_remove(s))

from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
stemmer = PorterStemmer()
def stemming(text):
    words = word_tokenize(text)
    new_txt = []
    for word in words:
        new_txt.append(stemmer.stem(word))
    txt = ' '.join(new_txt)
    return txt
data['report'] = data['report'].progress_apply(lambda s: stemming(s))

data.head()

,Unnamed: 0,edf_location,label,report
0,0,E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...,0,clinic histori 55yearold righthand woman cn va...
1,1,E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...,1,clinic histori 27yearold woman focal sclerosi ...
2,2,E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...,0,clinic histori 23yearold male histori seizur 2...
3,3,E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...,0,clinic histori 56 year old male small lacunar ...
4,4,E:/EEG DATA/train_ordered/02_tcp_le/Formatted/...,0,clinic histori 56 year old male small lacunar ...


In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
#data['report'] = data['report'].apply(lambda string: [string])
#data['report']=[" ".join(review) for review in data['report'].values]
# create the transform
vectorizer = TfidfVectorizer(max_df=0.9)
# tokenize and build vocab
vectorizer.fit(data['report'])
# encode document
vector = vectorizer.transform(data['report'])
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

(181, 3561)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vector, data['label'], test_size=0.2, random_state=1)

In [40]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
model=Sequential()
model.add(Embedding(vector.shape[1], 32, input_length = vector.shape[1]))
model.add(LSTM(500))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 3561, 32)          113952    
_________________________________________________________________
lstm_4 (LSTM)                (None, 500)               1066000   
_________________________________________________________________
dense_4 (Dense)              (None, 128)               64128     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
Total params: 1,244,209
Trainable params: 1,244,209
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
model.fit(X_train, y_train, epochs=5, batch_size=12, verbose = 1)

Epoch 1/5
144/144 [==============================] - ETA: 4:42 - loss: 0.6934 - acc: 0.333 - ETA: 4:22 - loss: 0.6938 - acc: 0.416 - ETA: 4:06 - loss: 0.6978 - acc: 0.416 - ETA: 3:35 - loss: 0.7029 - acc: 0.375 - ETA: 3:06 - loss: 0.7019 - acc: 0.383 - ETA: 2:38 - loss: 0.7011 - acc: 0.361 - ETA: 2:12 - loss: 0.7000 - acc: 0.381 - ETA: 1:45 - loss: 0.6982 - acc: 0.416 - ETA: 1:18 - loss: 0.6961 - acc: 0.444 - ETA: 52s - loss: 0.6961 - acc: 0.450 - ETA: 26s - loss: 0.6948 - acc: 0.46 - 319s 2s/step - loss: 0.6935 - acc: 0.4722
Epoch 2/5
144/144 [==============================] - ETA: 4:54 - loss: 0.5907 - acc: 0.750 - ETA: 4:28 - loss: 0.7127 - acc: 0.750 - ETA: 4:04 - loss: 0.6877 - acc: 0.722 - ETA: 3:35 - loss: 0.7025 - acc: 0.645 - ETA: 3:08 - loss: 0.7067 - acc: 0.600 - ETA: 2:43 - loss: 0.7026 - acc: 0.597 - ETA: 2:15 - loss: 0.7053 - acc: 0.559 - ETA: 1:48 - loss: 0.7027 - acc: 0.562 - ETA: 1:21 - loss: 0.7008 - acc: 0.564 - ETA: 54s - loss: 0.6986 - acc: 0.575 - ETA: 27s - loss:

In [19]:
result = model.evaluate(X_test, y_test)
print(result)

37/37 [==============================] - ETA:  - 15s 407ms/step
[0.6599593549161344, 0.6486486518705213]


In [43]:
#Now we let the model train on the entire data for better classification. 
#Re-compile the model first or the training is applied additionally to the current model.
model.fit(vector, data['label'], epochs = 7, batch_size = 12, verbose = 1)

Epoch 1/7
181/181 [==============================] - ETA: 5:54 - loss: 0.6932 - acc: 0.500 - ETA: 5:49 - loss: 0.6825 - acc: 0.583 - ETA: 5:14 - loss: 1.0458 - acc: 0.638 - ETA: 4:42 - loss: 0.9558 - acc: 0.687 - ETA: 4:14 - loss: 0.9017 - acc: 0.666 - ETA: 3:47 - loss: 0.8673 - acc: 0.638 - ETA: 3:28 - loss: 0.8412 - acc: 0.631 - ETA: 3:03 - loss: 0.8245 - acc: 0.604 - ETA: 2:38 - loss: 0.8101 - acc: 0.592 - ETA: 2:14 - loss: 0.7963 - acc: 0.608 - ETA: 1:47 - loss: 0.7860 - acc: 0.606 - ETA: 1:21 - loss: 0.7786 - acc: 0.597 - ETA: 54s - loss: 0.7713 - acc: 0.596 - ETA: 28s - loss: 0.7660 - acc: 0.58 - ETA: 2s - loss: 0.7596 - acc: 0.5944 - 401s 2s/step - loss: 0.7586 - acc: 0.5967
Epoch 2/7
181/181 [==============================] - ETA: 5:42 - loss: 0.6401 - acc: 0.666 - ETA: 5:57 - loss: 0.7580 - acc: 0.500 - ETA: 5:40 - loss: 0.7484 - acc: 0.444 - ETA: 5:14 - loss: 0.7226 - acc: 0.541 - ETA: 4:52 - loss: 0.7077 - acc: 0.583 - ETA: 4:31 - loss: 0.7065 - acc: 0.569 - ETA: 4:04 - loss

In [44]:
from keras.models import load_model
#Save the weights and architecture of the model
model.save('E:/EEG Analysis/CLASSIFIER/text_model.h5')

In [45]:

txt = punct_remove(txt)
print(txt)
txt  = stemming(txt)
print(txt)
vector = vectorizer.transform([txt])
res = model.predict(vector)
print(res)
print(model.metrics)

In [48]:
res = model.predict(vector)